In [1]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [2]:
tr_df = pd.read_clipboard()

In [3]:
tr_df

,Parent ID,DNA ID,Culture ID,Fragment ID,location,Concentration,ng (assuming 12.5 uL),Marker,Result Strain Sample ID
0,24943,36445,514175,514005,M20.25.2.34,67.8,847.50,URA-URA<pGRR-W20W36-yeGFP> (v2),36476
1,33810,36353,513865,514139,M20.26.6.58,24.4,305.00,HIS-HIS<7XTetO-pMinCyc1-URGR-W5>,36399
2,24943,36448,514175,514141,M20.26.8.55,48.2,602.50,URA-URA<pGRR-W5W36-yeGFP> (v2),36477
3,24942,36451,524273,514142,M20.26.8.62,50.5,631.25,URA-URA<pGRR-W10W17-yeGFP> (v2),36478
4,24940,36447,514174,514003,M20.25.2.32,124.9,1561.25,URA-URA<pGRR-W36W36-yeGFP> (v2),36479
5,24942,36446,524273,514006,M20.25.2.35,123.8,1547.50,URA-URA<pGRR-W17W17-yeGFP> (v2),36480
6,24942,36450,524273,514140,M20.26.6.59,31.1,388.75,URA-URA<pGRR-W10W20-yeGFP> (v2),36481
7,33810,36339,513865,514137,M20.26.6.56,15.8,197.50,HIS-HIS<7XTetO-pMinCyc1-URGR-W34>,36400
8,33810,36332,513865,514138,M20.26.6.57,22.2,277.50,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>,36401


In [4]:
tr_to_id = {}

for _, row in tr_df.iterrows():
    pid = row['Parent ID']
    did = row['DNA ID']
    key = (pid, did)
    tr_to_id[key] = row['Result Strain Sample ID']

In [27]:
import pandas as pd
import json
import re

df = pd.read_csv('build_plan.csv', index_col=0)
df.iloc[0]

rows = list(df.iterrows())
parents = []
dna = []
replacements = {}

def replacement_dnas(ids):
    new_ids = []
    for id in ids:
        dna_sample = aq.Sample.find(id)

        name = dna_sample.name
    #     if 'W5W36' in name:
    #         name = re.sub('W5W36', 'W36W5', name)
        replacement = aq.Sample.find_by_name(name + " (v2)")

        if replacement:
            replacements[dna_sample.id] = replacement.id
            dna_sample = replacement
            
        new_ids.append(dna_sample.id)
    return new_ids

for i, row in rows:
    parent = eval(row['parent'])
    
    if isinstance(parent, list):
        new_parents = []
        for p in parent:
            if p[0] == 33811:
                p = (33810, aq.Sample.find(33810).name)
            new_parents.append(p)
        parent = [p[0] for p in new_parents if 'TRASH' not in p[1]][0]
    elif isinstance(parent, tuple):
        parent = (parent[0], tuple(replacement_dnas(parent[1])))
    parents.append(parent)
    
    dna.append(replacement_dnas([row['dna']])[0])
        
df['parent'] = parents
df['dna'] = dna

results = []

for _, row in df.iterrows():
    r = eval(row['result'])
    key = (row['parent'], row['dna'])
    if key in tr_to_id:
        r = tr_to_id[key]
    else:
        if isinstance(r, tuple):
            dna_ids = []
            for _r in r[1]:
                if _r in replacements:
                    _r = replacements[_r]
                dna_ids.append(_r)
            r = (r[0], dna_ids)
        else:
            r = 'MISSING'
    results.append(r)
    
df['result'] = results

df.to_clipboard()
df

,step,parent,dna,result,weight,name,permutation,created_at,id,dna_name
0,1,33810,36332,36401,1,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>
1,2,36398,36469,"(Mat A, [1380, 33579, 36332, 36469])",4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,URA-URA<pGRR-W10W8-URGR-W5>
2,1,24943,36445,36476,4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,URA-URA<pGRR-W20W36-yeGFP>
3,2,"(Mat Alpha, (1096, 5533, 36445))",36470,"(Mat Alpha, [1096, 5533, 36470, 36445])",4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,LEU-LEU<pGRR-W5W5-URGR-W36>
4,3,"(Mat Alpha, (1096, 5533, 36470, 36445))",36482,"(Mat Alpha, [1096, 5533, 36482, 36470, 36445])",4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,ARS314-bleo<pGRR-W5W5-URGR-W20>
15,1,33810,36339,36400,1,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W34>
16,2,36397,36475,"(Mat A, [1380, 33579, 36339, 36475])",4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,URA-URA<pGRR-W8W34-URGR-W17>
17,1,24942,36450,36481,4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,URA-URA<pGRR-W10W20-yeGFP>
18,2,"(Mat Alpha, (1096, 5534, 36450))",36474,"(Mat Alpha, [1096, 5534, 36474, 36450])",4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,LEU-LEU<pGRR-W17W17-URGR-W20>
19,3,"(Mat Alpha, (1096, 5534, 36474, 36450))",36483,"(Mat Alpha, [1096, 5534, 36483, 36474, 36450])",4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,ARS314-bleo<pGRR-W17W17-URGR-W10>


In [13]:
for id in list(set(df[df.step == 1].result)):
    sample = aq.Sample.find(id)
    item = aq.Item.new(
        sample_id=sample.id,
        object_type_id = aq.ObjectType.find_by_name("Unverified Yeast Overnight Suspension").id
    )
    item.location = "30C Incubator"
    item.save()
    print(item.id)

514738
514739
514740
514741
514742
514743
514744
514745
514746


In [18]:
transformations = []

for _, row in df[df['step'] == 2].iterrows():
    transformations.append((row['parent'], row['dna']))
    
transformations = list(set(transformations))

transformations

[(('Mat Alpha', (1096, 5533, 36336)), 36470),
 (('Mat Alpha', (1096, 5534, 36343)), 36474),
 (36398, 36468),
 (36379, 36471),
 (('Mat Alpha', (1096, 5534, 36350)), 36473),
 (36398, 36469),
 (36397, 36472),
 (36397, 36475)]

In [20]:
df

,step,parent,dna,result,weight,name,permutation,created_at,id,dna_name
0,1,33810,36332,36401,1,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>
1,2,36398,36469,"(Mat A, [1380, 33579, 36332, 36469])",4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,URA-URA<pGRR-W10W8-URGR-W5>
2,1,24943,36445,36476,4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,URA-URA<pGRR-W20W36-yeGFP>
3,2,"(Mat Alpha, (1096, 5533, 36336))",36470,"(Mat Alpha, [1096, 5533, 36470, 36445])",4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,LEU-LEU<pGRR-W5W5-URGR-W36>
4,3,"(Mat Alpha, (1096, 5533, 36335, 36336))",36482,"(Mat Alpha, [1096, 5533, 36482, 36470, 36445])",4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,ARS314-bleo<pGRR-W5W5-URGR-W20>
15,1,33810,36339,36400,1,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W34>
16,2,36397,36475,"(Mat A, [1380, 33579, 36339, 36475])",4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,URA-URA<pGRR-W8W34-URGR-W17>
17,1,24942,36450,36481,4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,URA-URA<pGRR-W10W20-yeGFP>
18,2,"(Mat Alpha, (1096, 5534, 36343))",36474,"(Mat Alpha, [1096, 5534, 36474, 36450])",4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,LEU-LEU<pGRR-W17W17-URGR-W20>
19,3,"(Mat Alpha, (1096, 5534, 36342, 36343))",36483,"(Mat Alpha, [1096, 5534, 36483, 36474, 36450])",4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,ARS314-bleo<pGRR-W17W17-URGR-W10>


In [70]:
results = []

for _, row in df.iterrows():
    r = eval(row['result'])
    key = (row['parent'], row['dna'])
    if key in tr_to_id:
        r = tr_to_id[key]
    else:
        if isinstance(r, tuple):
            dna_ids = []
            for _r in r[1]:
                if _r in replacements:
                    _r = replacements[_r]
                dna_ids.append(_r)
            r = (r[0], dna_ids)
        else:
            r = 'MISSING'
    results.append(r)
    
df['result'] = results

df

TypeError: eval() arg 1 must be a string, bytes or code object